# <center>Traitement et nettoyage des données</center>

Dans ce notebook nous allons nettoyer nos données (texts) avant de les fournir à un mmodel de machine learning


In [ ]:
import pandas as pd
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv("drive/MyDrive/test.csv", dtype={"headline": str})

In [ ]:
dataset

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [ ]:
dataset['comment_text'][0]

"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

------

In [ ]:
import subprocess

subprocess.run(["pip", "install","-U","gensim"])
subprocess.run(["pip", "install","-U","unidecode"])

CompletedProcess(args=['pip', 'install', '-U', 'unidecode'], returncode=0)

In [ ]:
import itertools
import os
import re
import secrets
import string

import pandas as pd
import spacy
from unidecode import unidecode

from itertools import chain
from gensim.utils import simple_preprocess

On commence par vérifier la qualité des données c'est à dire qu'on s'assure que nos données sont bien des chaines de caractères

In [ ]:
def check_data_quality(texts):
    temp = itertools.chain.from_iterable(texts)
    pandas_dataframe = pd.Series(temp)
    quality = True
    for value in pandas_dataframe.isna():
      quality = quality and (value == False)
    return quality

In [ ]:
print(f"Is the dataset passing our data quality check?\n{check_data_quality(dataset["comment_text"])}")

Is the dataset passing our data quality check?
True


Par la suite on met le texte en miniscule, on enlève les retours à la ligne, les chiffres, les extensions des images, les urls, la punctuation et les accents

In [ ]:
def filter_text(texts_in):
    compteur = 0
    for values in texts_in["comment_text"]:
      temp = values.lower()
      new_s = re.sub(r'(\n)|([0-9]+)|(.jpg)|(.bmp)|(.jpeg)|(.png)|(https?://[^\s]+)',' ',temp)
      table = str.maketrans({p: ' ' for p in string.punctuation})
      new_text = new_s.translate(table)
      final = unidecode(new_text)
      texts_in.loc[compteur, "comment_text"] = final
      compteur = compteur + 1
    return texts_in


In [ ]:
copy_dataset = dataset

In [ ]:
texts = filter_text(copy_dataset)


In [ ]:
texts['comment_text'][0]

'yo bitch ja rule is more succesful then you ll ever be whats up with you and hating you sad mofuckas   i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me  ja rule is about pride in da music man  dont diss that shit on him  and nothin is wrong bein like tupac he was a brother too   fuckin white boys get things right next time  '

Par la suite nous avons converti nos phrases en token.

In [ ]:
def sent_to_words(sentences):
    return simple_preprocess(sentences,max_len=800)


In [ ]:
unify_texts = []
for substance in texts['comment_text']:
  unify_texts.append(list(sent_to_words(substance)))

In [ ]:
unify_texts

[['yo',
  'bitch',
  'ja',
  'rule',
  'is',
  'more',
  'succesful',
  'then',
  'you',
  'll',
  'ever',
  'be',
  'whats',
  'up',
  'with',
  'you',
  'and',
  'hating',
  'you',
  'sad',
  'mofuckas',
  'should',
  'bitch',
  'slap',
  'ur',
  'pethedic',
  'white',
  'faces',
  'and',
  'get',
  'you',
  'to',
  'kiss',
  'my',
  'ass',
  'you',
  'guys',
  'sicken',
  'me',
  'ja',
  'rule',
  'is',
  'about',
  'pride',
  'in',
  'da',
  'music',
  'man',
  'dont',
  'diss',
  'that',
  'shit',
  'on',
  'him',
  'and',
  'nothin',
  'is',
  'wrong',
  'bein',
  'like',
  'tupac',
  'he',
  'was',
  'brother',
  'too',
  'fuckin',
  'white',
  'boys',
  'get',
  'things',
  'right',
  'next',
  'time'],
 ['from', 'rfc', 'the', 'title', 'is', 'fine', 'as', 'it', 'is', 'imo'],
 ['sources', 'zawe', 'ashton', 'on', 'lapland'],
 ['if',
  'you',
  'have',
  'look',
  'back',
  'at',
  'the',
  'source',
  'the',
  'information',
  'updated',
  'was',
  'the',
  'correct',
  'form',
 

J'ai crée un fichier dans le quel j'ai rassemblé tous les "stop words" les plus utilisé en français et en anglais. Je l'utilise par la suite pour nettoyer mes données


In [ ]:
def get_stopwords(path):
    stopwords = pd.read_csv(path)
    return stopwords


In [ ]:
stopwords = get_stopwords("drive/MyDrive/stopwords.txt")
stop_words = set(stopwords.iloc[:, 0].astype(str))

In [ ]:
stop_words

{'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 'aint',
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 'arent',
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 'came',
 'can',
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'cmon',
 'co',
 'com',
 'come',
 'comes',
 'concerning',
 'consequentl

In [ ]:
#stopwords = get_stopwords()
def remove_stopwords(tokens, stop_words):
    return [w for w in tokens if w not in stop_words]

In [ ]:
hello = []
for token in unify_texts:
  hello.append(remove_stopwords(token,stop_words))

In [ ]:
final_data = hello

In [ ]:
len(final_data)

159571

In [ ]:
count = 0
for tokens in final_data:
  info = " ".join(tokens)
  copy_dataset.loc[count, "comment_text"] = info
  count =count + 1

In [ ]:
copy_dataset

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule succesful ll hating sad mofuc...
1,0000247867823ef7,rfc title fine imo
2,00013b17ad220c46,sources zawe ashton lapland
3,00017563c3f7919a,back source information updated correct form g...
4,00017695ad8997eb,don anonymously edit articles
...,...,...
153159,fffcd0960ee309b5,totally agree stuff long crap
153160,fffd7a9a6eb32c16,throw field home plate faster throwing cut man...
153161,fffda9e8d6fafa9e,okinotorishima categories agree correct confus...
153162,fffe8f1340a79fc2,founding nations germany law return similar is...


In [ ]:
copy_dataset.to_csv("cleaned_test.csv", index=False)
